## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-18-06-49-31 +0000,bbc,Chinese Nobel laureate and physicist Chen Ning...,https://www.bbc.com/news/articles/cdxrzzk02plo...
1,2025-10-18-06-46-14 +0000,nyt,Trump Backs Off Tomahawk Suggestion After Zele...,https://www.nytimes.com/live/2025/10/17/us/tru...
2,2025-10-18-06-30-01 +0000,bbc,Exciting results from blood test for 50 cancers,https://www.bbc.com/news/articles/c205g21n1zzo...
3,2025-10-18-06-28-53 +0000,bbc,Israel confirms latest body returned from Gaza...,https://www.bbc.com/news/articles/c5yp3eq9knvo...
4,2025-10-18-06-26-44 +0000,bbc,Who are the released hostages?,https://www.bbc.com/news/articles/cpvl9k4mw8no...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,54
227,new,18
177,ukraine,15
153,government,12
237,andrew,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
48,2025-10-18-00-06-00 +0000,wsj,President Trump has commuted the sentence of f...,https://www.wsj.com/politics/trump-commutes-se...,110
21,2025-10-18-03-30-12 +0000,nypost,Disgraced ex-Long Island Rep. George Santos re...,https://nypost.com/2025/10/17/us-news/george-s...,108
134,2025-10-17-20-49-26 +0000,nypost,"Trump urges Russia, Ukraine to end war ‘where ...",https://nypost.com/2025/10/17/us-news/trump-sa...,105
58,2025-10-17-23-45-00 +0000,wsj,Trump Says He Would Rather End War Than Send T...,https://www.wsj.com/world/trump-says-hed-rathe...,99
91,2025-10-17-22-26-18 +0000,nypost,Trump orders lying ex-Long Island Rep. George ...,https://nypost.com/2025/10/17/us-news/trump-or...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
48,110,2025-10-18-00-06-00 +0000,wsj,President Trump has commuted the sentence of f...,https://www.wsj.com/politics/trump-commutes-se...
174,69,2025-10-17-18-19-43 +0000,nypost,Jeffrey Epstein-linked Prince Andrew forced to...,https://nypost.com/2025/10/17/world-news/princ...
210,54,2025-10-17-16-42-19 +0000,nypost,John Bolton spotted still using White House ba...,https://nypost.com/2025/10/17/us-news/john-bol...
150,53,2025-10-17-19-56-00 +0000,wsj,Bank of Canada Gov. Tiff Macklem said data nex...,https://www.wsj.com/articles/bank-of-canada-su...
134,51,2025-10-17-20-49-26 +0000,nypost,"Trump urges Russia, Ukraine to end war ‘where ...",https://nypost.com/2025/10/17/us-news/trump-sa...
311,50,2025-10-17-10-00-00 +0000,wapo,"The average new-car price hit a record $50,000...",https://www.washingtonpost.com/business/2025/1...
327,43,2025-10-17-09-04-01 +0000,nyt,What We Know About the Latest ‘No Kings’ Prote...,https://www.nytimes.com/2025/10/17/us/no-kings...
242,43,2025-10-17-15-18-45 +0000,nyt,"Baek Sehee, South Korean Author of ‘I Want to ...",https://www.nytimes.com/2025/10/17/arts/baek-s...
54,42,2025-10-17-23-50-52 +0000,nypost,Salesforce CEO Marc Benioff apologizes for say...,https://nypost.com/2025/10/17/us-news/salesfor...
315,34,2025-10-17-09-59-55 +0000,cbc,Madagascar's military coup leader is sworn in ...,https://www.cbc.ca/news/world/madagascar-milit...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
